In [1]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.PipelineModel
import org.apache.spark.ml.classification.NaiveBayes
import org.apache.spark.ml.classification.NaiveBayesModel
import org.apache.spark.ml.feature.{RegexTokenizer,StopWordsRemover,NGram,HashingTF,VectorAssembler,StringIndexer,IndexToString}
import org.apache.spark.ml.tuning.{ParamGridBuilder, CrossValidator}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.mllib.util.MLUtils

import org.apache.spark.sql.Row
import org.apache.spark.sql.DataFrame
import org.apache.spark.storage.StorageLevel

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1607312812821_0006,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.PipelineModel
import org.apache.spark.ml.classification.NaiveBayes
import org.apache.spark.ml.classification.NaiveBayesModel
import org.apache.spark.ml.feature.{RegexTokenizer, StopWordsRemover, NGram, HashingTF, VectorAssembler, StringIndexer, IndexToString}
import org.apache.spark.ml.tuning.{ParamGridBuilder, CrossValidator}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.sql.Row
import org.apache.spark.sql.DataFrame
import org.apache.spark.storage.StorageLevel


In [2]:
val reviews = "s3://bd-project-data/review.json.bz2"
var reviewsDF = spark.sqlContext.read.json(reviews)
reviewsDF.registerTempTable("reviews")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

reviews: String = s3://bd-project-data/review.json.bz2
reviewsDF: org.apache.spark.sql.DataFrame = [business_id: string, cool: bigint ... 7 more fields]


In [3]:
reviewsDF.groupBy("stars").count().orderBy("stars").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------+
|stars|  count|
+-----+-------+
|  1.0|1283897|
|  2.0| 635072|
|  3.0| 842289|
|  4.0|1673404|
|  5.0|3586460|
+-----+-------+



In [4]:
reviewsDF = reviewsDF.sample(withReplacement = false,0.2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

reviewsDF: org.apache.spark.sql.DataFrame = [business_id: string, cool: bigint ... 7 more fields]


In [8]:
var reviewsDF = spark.sqlContext.sql(
s"""
  SELECT text, label, rowNumber FROM (
    SELECT
       CASE
              WHEN reviews.stars  = 1 THEN 1
              WHEN reviews.stars  = 2 THEN 1
              WHEN reviews.stars  = 3 THEN 2
              WHEN reviews.stars  = 4 THEN 2
              WHEN reviews.stars  = 5 THEN 2
       END AS label
      ,reviews.text AS text
      ,row_number() OVER (PARTITION BY stars ORDER BY rand()) AS rowNumber
    FROM reviews
  ) reviews
  WHERE rowNumber <= 60000
  """
)

reviewsDF.persist(org.apache.spark.storage.StorageLevel.MEMORY_AND_DISK)

reviewsDF.groupBy("label").count().orderBy("label").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

reviewsDF: org.apache.spark.sql.DataFrame = [text: string, label: int ... 1 more field]
res9: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [text: string, label: int ... 1 more field]
+-----+------+
|label| count|
+-----+------+
|    1|120000|
|    2|180000|
+-----+------+



In [10]:
val training = reviewsDF.filter(reviewsDF("rowNumber") <= 50000).select("text","label")
val test = reviewsDF.filter(reviewsDF("rowNumber") > 50000).select("text","label")

training.persist(org.apache.spark.storage.StorageLevel.MEMORY_AND_DISK)
test.persist(org.apache.spark.storage.StorageLevel.MEMORY_AND_DISK)

val numTraining = training.count()
val numTest = test.count()

println(s"numTraining = $numTraining, numTest = $numTest")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

training: org.apache.spark.sql.DataFrame = [text: string, label: int]
test: org.apache.spark.sql.DataFrame = [text: string, label: int]
res19: training.type = [text: string, label: int]
res20: test.type = [text: string, label: int]
numTraining: Long = 250000
numTest: Long = 50000
numTraining = 250000, numTest = 50000


## Defining Parts of the Pipeline

In [11]:
val regexTokenizer = { new RegexTokenizer()
  .setPattern("[a-zA-Z']+")
  .setGaps(false)
  .setInputCol("text")
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

regexTokenizer: org.apache.spark.ml.feature.RegexTokenizer = regexTok_f3a30e2c4c1d


In [12]:
val remover = { new StopWordsRemover()
  .setInputCol(regexTokenizer.getOutputCol)
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

remover: org.apache.spark.ml.feature.StopWordsRemover = stopWords_e07341cfa475


In [13]:
val ngram2 = { new NGram()
  .setInputCol(remover.getOutputCol)
  .setN(2)
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ngram2: org.apache.spark.ml.feature.NGram = ngram_4317578f3930


In [14]:
val ngram3 = { new NGram()
  .setInputCol(remover.getOutputCol)
  .setN(3)
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ngram3: org.apache.spark.ml.feature.NGram = ngram_2ac9fe31f83d


In [15]:
val removerHashingTF = { new HashingTF()
  .setInputCol(remover.getOutputCol)
}
val ngram2HashingTF = { new HashingTF()
  .setInputCol(ngram2.getOutputCol)
}
val ngram3HashingTF = { new HashingTF()
  .setInputCol(ngram3.getOutputCol)
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

removerHashingTF: org.apache.spark.ml.feature.HashingTF = hashingTF_bfa2a7da87d8
ngram2HashingTF: org.apache.spark.ml.feature.HashingTF = hashingTF_dc57b258b1dc
ngram3HashingTF: org.apache.spark.ml.feature.HashingTF = hashingTF_bc55afd824f7


In [16]:
val assembler = { new VectorAssembler()
  .setInputCols(Array(removerHashingTF.getOutputCol, ngram2HashingTF.getOutputCol, ngram3HashingTF.getOutputCol))
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_60e693c6d4dc


In [17]:
val labelIndexer = { new StringIndexer()
  .setInputCol("label")
  .setOutputCol("indexedLabel")
  .fit(reviewsDF)
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

labelIndexer: org.apache.spark.ml.feature.StringIndexerModel = strIdx_50584bec69e7


In [18]:
val labelConverter = { new IndexToString()
  .setInputCol("prediction")
  .setOutputCol("predictedLabel")
  .setLabels(labelIndexer.labels)
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

labelConverter: org.apache.spark.ml.feature.IndexToString = idxToStr_6ac929c49b5d


In [19]:
val nb = { new NaiveBayes()
  .setLabelCol(labelIndexer.getOutputCol)
  .setFeaturesCol(assembler.getOutputCol)
  .setPredictionCol("prediction")
  .setModelType("multinomial")
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

nb: org.apache.spark.ml.classification.NaiveBayes = nb_f2ce5794ed9b


## Defining The Pipeline

In [20]:
val pipeline = { new Pipeline()
  .setStages(Array(regexTokenizer, remover, ngram2, ngram3, removerHashingTF, ngram2HashingTF, ngram3HashingTF, assembler, labelIndexer, nb, labelConverter))
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

pipeline: org.apache.spark.ml.Pipeline = pipeline_7e13612dc3e8


In [21]:
val paramGrid = { new ParamGridBuilder()
  .addGrid(removerHashingTF.numFeatures, Array(1000,10000))
  .addGrid(ngram2HashingTF.numFeatures, Array(1000,10000))
  .addGrid(ngram3HashingTF.numFeatures, Array(1000,10000))
  .build()
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	hashingTF_dc57b258b1dc-numFeatures: 1000,
	hashingTF_bc55afd824f7-numFeatures: 1000,
	hashingTF_bfa2a7da87d8-numFeatures: 1000
}, {
	hashingTF_dc57b258b1dc-numFeatures: 10000,
	hashingTF_bc55afd824f7-numFeatures: 1000,
	hashingTF_bfa2a7da87d8-numFeatures: 1000
}, {
	hashingTF_dc57b258b1dc-numFeatures: 1000,
	hashingTF_bc55afd824f7-numFeatures: 10000,
	hashingTF_bfa2a7da87d8-numFeatures: 1000
}, {
	hashingTF_dc57b258b1dc-numFeatures: 10000,
	hashingTF_bc55afd824f7-numFeatures: 10000,
	hashingTF_bfa2a7da87d8-numFeatures: 1000
}, {
	hashingTF_dc57b258b1dc-numFeatures: 1000,
	hashingTF_bc55afd824f7-numFeatures: 1000,
	hashingTF_bfa2a7da87d8-numFeatures: 10000
}, {
	hashingTF_dc57b258b1dc-numFeatures: 10000,
	hashingTF_bc55afd82...

In [26]:
val cv = { new CrossValidator()
  .setEstimator(pipeline)
  .setEvaluator(new MulticlassClassificationEvaluator().setLabelCol("indexedLabel").setPredictionCol("prediction").setMetricName("accuracy"))
  .setEstimatorParamMaps(paramGrid)
  .setNumFolds(3) // Use 3+ in practice
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

cv: org.apache.spark.ml.tuning.CrossValidator = cv_6afd0aa9a2be


In [28]:
val cvModel = cv.fit(training)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

cvModel: org.apache.spark.ml.tuning.CrossValidatorModel = cv_6afd0aa9a2be


In [29]:
val predictions = cvModel.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.sql.DataFrame = [text: string, label: int ... 13 more fields]


In [31]:
val evaluator = { new MulticlassClassificationEvaluator()
  .setLabelCol("indexedLabel")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
}
val precision = evaluator.evaluate(predictions)
println("Test Error = " + (1.0 - precision))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_7f327036dd51
precision: Double = 0.80634
Test Error = 0.19366000000000005
